## Modules

In [1]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
from itertools import product
import numpy as np
import pandas as pd

from time import time

from tdcalendar import *
from tdcalendar.hkcalendar import *

## Handling trading days

### Holiday files

In [ ]:
# US stock market holidays
with open(os.path.join('holiday_ny.csv'), 'r') as f:
    holidaylines_ny = f.readlines()
    holidaylist_ny = [datetime.strptime(row.split(',')[0], '%Y-%m-%d') for row in holidaylines_ny[1:]]

# HK stock market holidays
with open(os.path.join('holiday_hk.csv'), 'r') as f:
    holidaylines_hk = f.readlines()
    holidaylist_hk = [datetime.strptime(row.split(',')[0], '%Y-%m-%d') for row in holidaylines_hk[1:]]

### Trading days

In [ ]:
def getwkdays(startyr=1997, endyr=2046, form='%Y-%m-%d'):
    """Get all working days (non-weekend) to string in a list."""
    dtlist = []
    date = datetime.strptime(f'{startyr}-01-01', '%Y-%m-%d')
    while (date.year >= startyr) and (date.year <= endyr):
        if date.weekday() <= 4:
            dtlist.append(date.strftime(form))
        date += timedelta(days=1)

    return dtlist

workdtlist0 = getwkdays()  #  Format yyyy-mm-dd
workdtlist1 = getwkdays(form='%Y%m%d')  # Format yyyymmdd
workdtlist2 = getwkdays(form='%y%m%d')  # Format yymmdd

In [ ]:
def gettradedays(holidaylist, startdt=datetime(2020, 1, 1), enddt=datetime(2022, 12, 31), form='%Y-%m-%d'):
    """Get all trading day spanning a period, excluding holidays."""
    if form == '%Y-%m-%d':
        workdtlist = workdtlist0
    elif form == '%Y%m%d':
        workdtlist = workdtlist1
    elif form == '%y%m%d':
        workdtlist = workdtlist2
    else:
        workdtlist = getwkdays(form=form)
        
    startstr = startdt.strftime(form)
    endstr = enddt.strftime(form)
    holidaystrlist = [date.strftime(form) for date in holidaylist]
    tdlist = [dtstr for dtstr in workdtlist if (dtstr >= startstr) and (dtstr <= endstr)]
    tdlist = [dtstr for dtstr in tdlist if dtstr not in holidaystrlist]

    return tdlist

In [ ]:
tdaylist_hk1 = gettradedays(holidaylist_hk)
print(tdaylist_hk1[-20:])

### Get latest trading day

In [ ]:
def getlatesttradingday(holidaylist, offset=6, form='%Y-%m-%d'):
    """Obtain the latest trading date."""
    today = datetime.today() - timedelta(days=1, hours=offset)
    earlyday = today - timedelta(days=30)
    tdlist = [dtstr for dtstr in gettradedays(holidaylist, earlyday, today, form)]

    return tdlist[-1]

In [ ]:
print(getlatesttradingday(holidaylist_ny))
print(getlatesttradingday(holidaylist_hk))

### HK Futures monthly settlement days

In [ ]:
def gethkexpiry(monthstr='JAN-21'):
    """Obtain HK monthly expiry day in a month."""
    monthstart = datetime.strptime(monthstr, '%b-%y')
    monthstr = monthstart.strftime('%Y-%m')
    year = monthstart.year
    tdaylist = gettradedays(holidaylist_hk, datetime(year, 1, 1), datetime(year, 12, 31))
    tdaylist = [dtstr for dtstr in tdaylist if dtstr[:7] == monthstr] 
    
    return tdaylist[-2]

starttime = time()

expirydict = {}

for year, month in product(range(2007, 2024), range(1, 13)):
    monthstr = (datetime(year, month, 1).strftime('%b-%y')).upper()
    expirydict[monthstr] = gethkexpiry(monthstr)
    
expirylist = list(expirydict.values())
expirylist.sort()

endtime = time()
print(f'Time elapsed getting expiry date dict: {round(endtime - starttime, 4)}s')

### Read HK settlement day dictionary

In [ ]:
starttime = time()

with open('monthexpiry_hk.csv', 'r') as f:
    hkexpirylines = f.readlines()
    hkexpirydict = {}
    for row in hkexpirylines:
        monthstart = datetime.strptime(row[:7], '%Y-%m')
        monthstr = monthstart.strftime('%b-%y').upper()
        hkexpirydict[monthstr] = row[:-1]
        
hkexpirylist = list(hkexpirydict.values())
hkexpirylist.sort()
    
endtime = time()
print(f'Time elapsed getting expiry date dict: {round(endtime - starttime, 4)}s')


In [3]:
hkexpirydict

{'JAN-07': '2007-01-30',
 'FEB-07': '2007-02-27',
 'MAR-07': '2007-03-29',
 'APR-07': '2007-04-27',
 'MAY-07': '2007-05-30',
 'JUN-07': '2007-06-28',
 'JUL-07': '2007-07-30',
 'AUG-07': '2007-08-30',
 'SEP-07': '2007-09-27',
 'OCT-07': '2007-10-30',
 'NOV-07': '2007-11-29',
 'DEC-07': '2007-12-28',
 'JAN-08': '2008-01-30',
 'FEB-08': '2008-02-28',
 'MAR-08': '2008-03-28',
 'APR-08': '2008-04-29',
 'MAY-08': '2008-05-29',
 'JUN-08': '2008-06-27',
 'JUL-08': '2008-07-30',
 'AUG-08': '2008-08-28',
 'SEP-08': '2008-09-29',
 'OCT-08': '2008-10-30',
 'NOV-08': '2008-11-27',
 'DEC-08': '2008-12-30',
 'JAN-09': '2009-01-29',
 'FEB-09': '2009-02-26',
 'MAR-09': '2009-03-30',
 'APR-09': '2009-04-29',
 'MAY-09': '2009-05-27',
 'JUN-09': '2009-06-29',
 'JUL-09': '2009-07-30',
 'AUG-09': '2009-08-28',
 'SEP-09': '2009-09-29',
 'OCT-09': '2009-10-29',
 'NOV-09': '2009-11-27',
 'DEC-09': '2009-12-30',
 'JAN-10': '2010-01-28',
 'FEB-10': '2010-02-25',
 'MAR-10': '2010-03-30',
 'APR-10': '2010-04-29',
